In [1]:
!nvidia-smi

Mon Nov 30 18:12:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 12.6MB/s 
     |████████████████████████████████| 2.9MB 48.1MB/s 
     |████████████████████████████████| 890kB 45.2MB/s 
     |████████████████████████████████| 1.1MB 45.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c71da3d4db7ebbda6e9e15c5aa5afab41472720c5b79633a492501fe1202313b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import re
import json
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
import numpy as np

In [ ]:
from google.colab import files
# manually selected the shakespeare train and test files
train_path = files.upload()
test_path = files.upload()


Saving train.txt to train.txt


Saving test.txt to test.txt


## Preprocess data

In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, GPT2LMHeadModel, set_seed

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
train_path = 'train.txt'
test_path = 'test.txt'
set_seed(0)

## Prepare the data for our model

In [ ]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path, 
                                                        test_path, 
                                                        tokenizer)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:44: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


## Initialize model

In [ ]:
model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-trump", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 20, # Number of update steps between two evaluations.
    save_steps=40, # after # steps model is saved 
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
    )

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


##Train and Save model

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=460, training_loss=2.750154976222826)

In [ ]:
trainer.save_model()

##Sampling methods


In [ ]:
input = "What is the meaning of life "

In [ ]:
enc_input = tokenizer.encode(input, return_tensors='pt')
model = GPT2LMHeadModel.from_pretrained("./gpt2-trump", pad_token_id=tokenizer.eos_token_id)

In [ ]:
# greedy output
greedy_gen = model.generate(enc_input,
                            max_length=100
                            )
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_gen[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
What is the meaning of life  \
    That you, my lord, have so much to do with me?\
    I have no power to do with you.\
    I have no power to do with you.\
    I have no power to do with you.\
    I have no power to do with you.\
    I have no power to do with you.\
  


In [ ]:
# beam output
beam_gen = model.generate(enc_input,
                          max_length=100,
                          num_beams=5,
                          early_stopping=True
                          )
print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_gen[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
What is the meaning of life  \
    That you do not know it?\
\
  Rom. I do not know it.\
\
  Jul. I do not know it.\
\
  Rom. I do not know it.\
                                        


In [ ]:
# beam output with n-gram penalty and potentially all sequences returned for illustration purpose
# we observe almost no difference, if at all
beam_penalty_gen = model.generate(enc_input,
                          max_length=100,
                          num_beams=3,
                          no_repeat_ngram_size=2,
                          #num_return_sequences=3, 
                          early_stopping=True
                          )
# we could uncomment num_return_sequences to illustrate the different beams with virtually no difference
# print("Output:\n" + 100 * '-')
# for i, beam in enumerate(beam_penalty_gen):
#   print(i, tokenizer.decode(beam, skip_special_tokens=True))
print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_penalty_gen[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
What is the meaning of life?"\
  Ham. I do not know, my lord, but I think it is a question of the nature of things.   \
Exit. [Exeunt Hamlet and Guildenstern.]  O, what is this, that, when I speak to you, I say, 'I am a man'?  [Aside.] 'A man,' you say? 'Tis not a word, sir.'  Grief,


In [ ]:
# conditional sampling
sampling = model.generate(enc_input,
                          max_length=100,
                          do_sample=True,
                          top_k=0
                          )
print("Output:\n" + 100 * '-')
print(tokenizer.decode(sampling[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
What is the meaning of life  \
    In ten wars and up to five thousand wars? With the stars in the night?\
  MACBETH. They saw us.\
  DUNCAN. For a man, death cannot stand for days.\
  MACBETH. As if Romeo were a beetle-\
    Scouring this beetle's cave with his boar's ear. Go yonder?\
  MACBETH.


In [ ]:
# conditional sampling with temperature
temperature_sampling = model.generate(enc_input,
                          max_length=100,
                          do_sample=True,
                          top_k=0,
                          temperature=0.6
                          )
print("Output:\n" + 100 * '-')
print(tokenizer.decode(temperature_sampling[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
What is the meaning of life  \
    That you do not know the word\
    With which you speak?\
  DUNCAN. I know the word.\
  MALCOLM. So you know it well.\
  MACBETH. I know the word well.\
  DUNCAN. Gentlemen, I'll tell you the matter.\
    For we have a feeling\
   


In [ ]:
# top-k sampling where probability mass is redistibuted among top-k samples, and then next word is drawn from that distribution
top_k_sampling = model.generate(enc_input,
                          max_length=100,
                          do_sample=True,
                          top_k=40  # same as standard gpt2
                          )
print("Output:\n" + 100 * '-')
print(tokenizer.decode(top_k_sampling[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
What is the meaning of life  \
How many sorrows this world shall bear,\
Who will be buried in this earth?\
    O, what joys this very earth!\
    I can not, though it is heaven,\
    Not to hear of the ungainst man,\
    But to say I was in awe of heaven\
    When I saw him come to speak\
 


In [ ]:
# top-p sampling similar to top-k, but sampling from smallest possible set with bigger cumulative dist. than p
# then redistribute probability mass again and sample accordingly
top_p_sampling = model.generate(enc_input,
                          max_length=100,
                          do_sample=True,
                          top_p=0.9,
                          top_k=0
                          )
print("Output:\n" + 100 * '-')
print(tokenizer.decode(top_p_sampling[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
What is the meaning of life  \
That grows stale even within?\
\
COMINIUS.\
Not we.\
\
MARTINIUS.\
Let it not be, but my slave I should die.\
I am rather here-\
So should thou art here!\
Let my mutinous folly be-    To live here I must die.\
'Tis not too late.\
\



In [ ]:
# combination of top-p sampling and top-k to avoid very low ranked words while still allowing for dynamic selection
top_pk_combination_sampling = model.generate(enc_input,
                          max_length=100,
                          do_sample=True,
                          top_p=0.95,
                          top_k=40
                          )
print("Output:\n" + 100 * '-')
print(tokenizer.decode(top_pk_combination_sampling[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
What is the meaning of life  \
To tell us what our true desires are?\
\
MENENIUS.\
I have heard the good old man talk of 'tis.\
\
MARCIUS.\
What's your pleasure?\
\
MENENIUS.\
Th' use to me at once. I should have thought it in the time of\
The Roman wars;\
And, as we say, in the


In [ ]:
trainer.evaluate()

{'epoch': 10.0, 'eval_loss': 3.206249237060547}